In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
optimize_zero_metabolites.py

Iterate through a set of COBRA models, close specified exchange metabolites,
perform slim_optimize, and collect results into a CSV file.
Paths for input/output are hard-coded in main().
"""
import os
import pandas as pd
import cobra

# List of exchange reactions to close (set to zero)
METABOLITES_TO_ZERO = [
    "EX_ca2_e", "EX_cl_e", "EX_k_e", "EX_mg2_e", "EX_mn2_e",
    "EX_zn2_e", "EX_cobalt2_e", "EX_cu2_e", "EX_fe2_e",
    "EX_pi_e", "EX_nh4_e", "EX_so4_e", "EX_sel_e",
    "EX_mobd_e", "EX_ni2_e", "EX_na1_e"
]

def optimize_model(model: cobra.Model, metabolite: str) -> float:
    """
    Close one exchange metabolite and set glucose uptake, then run slim_optimize.
    """
    original_medium = model.medium.copy()
    modified = original_medium.copy()
    # ensure glucose uptake is open
    modified["EX_glc__D_e"] = 10
    # close the target exchange reaction
    modified[metabolite] = 0
    model.medium = modified

    result = model.slim_optimize()
    # restore original medium
    model.medium = original_medium
    return result

def main():
    # Hard-coded paths
    input_dir  = r'path/to/output/models'
    output_dir = r'path/to/output'
    output_csv = "data.csv"

    # create output folder if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # list all SBML model files
    xml_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.xml')]
    model_names = [os.path.splitext(f)[0] for f in xml_files]

    # prepare results DataFrame
    df = pd.DataFrame({'Metabolite': METABOLITES_TO_ZERO})
    for name in model_names:
        df[name] = 0.0

    # process each model
    for xml_file, model_name in zip(xml_files, model_names):
        path = os.path.join(input_dir, xml_file)
        try:
            model = cobra.io.read_sbml_model(path)
        except Exception as e:
            print(f"Failed to load model {xml_file}: {e}")
            continue

        results = []
        for met in METABOLITES_TO_ZERO:
            try:
                val = optimize_model(model, met)
            except Exception as e:
                print(f"Error optimizing {met} in {xml_file}: {e}")
                val = 0.0
            results.append(val)

        df[model_name] = results

    # write results to CSV
    out_path = os.path.join(output_dir, output_csv)
    df.to_csv(out_path, index=False, float_format="%.6f")
    print(f"Results saved to {out_path}")

if __name__ == "__main__":
    main()